# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [1]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Using TensorFlow backend.


17465344/17464789 [==============================] - 0s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


In [2]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 14, 2062, 20, 517, 125, 680, 5, 13, 104, 2062, 238, 30, 6, 2, 305, 7, 4, 644, 2, 2888, 1188, 14, 2062, 47, 684, 3531, 362, 39, 32, 638, 4, 2, 5, 166, 98, 1941, 6, 227, 40, 105, 39, 45, 6, 392, 182, 13, 482, 50, 26, 57, 506, 2, 4461, 23, 4, 916, 2, 1707, 23, 63, 29, 456, 45, 684, 24, 4, 2397, 2, 5, 24, 23, 703, 600, 7, 134, 362, 306, 55, 654, 5, 13, 828, 1786, 8, 67, 2, 989, 11, 5, 2232, 4, 2, 151, 13, 482, 18, 49, 7, 4, 840, 182, 362, 134, 780, 2, 71, 382, 35, 4893, 120, 719, 2, 13, 252, 440, 15, 32, 36, 81, 9, 1941, 5, 97, 4153, 10, 10, 95, 4, 136, 2, 1443, 8, 609, 121, 775, 5, 775, 7, 2624, 837, 44, 40, 36, 26, 11, 6, 2, 2, 621, 14, 253, 11, 276, 8, 6, 570, 34, 4168, 37, 4004, 31, 7, 98, 3063, 8, 140, 8, 703, 8, 2457, 982, 1276, 13, 197, 14, 20, 460, 122, 15, 4, 2113, 5, 27, 2, 26, 165, 243, 7, 1036, 40, 893, 538, 39, 4, 4355, 3442, 21, 19, 646, 445, 2, 42, 48, 25, 26, 2, 2, 40, 6, 2, 2377, 19, 6, 2, 10, 10, 820, 6, 3460, 2, 2662, 554, 773, 2, 6, 604, 7, 2, 362, 37

In [6]:
lenghts = []
for i in range(len(X_train)):
    lenghts.append(len(X_train[i]))

Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [3]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

1646592/1641221 [==============================] - 0s 0us/step
--- Review (with words) ---
['the', 'as', 'keaton', 'on', 'gave', 'better', 'call', 'to', 'was', 'two', 'keaton', '2', 'at', 'is', 'and', 'john', 'br', 'of', 'ridiculous', 'and', 'sidney', 'accent', 'as', 'keaton', 'there', 'events', 'press', 'production', 'or', 'an', 'light', 'of', 'and', 'to', 'find', 'any', 'conflict', 'is', 'far', 'just', 'films', 'or', 'if', 'is', 'playing', 'young', 'was', 'lead', 'more', 'he', 'even', 'flick', 'and', 'shake', 'are', 'of', 'laughs', 'and', 'warning', 'are', 'really', 'all', 'care', 'if', 'events', 'his', 'of', 'handled', 'and', 'to', 'his', 'are', 'cheap', 'score', 'br', 'while', 'production', 'himself', 'time', 'wish', 'to', 'was', 'general', 'torture', 'in', 'can', 'and', 'gay', 'this', 'to', 'portrays', 'of', 'and', 'old', 'was', 'lead', 'but', 'good', 'br', 'of', 'particular', 'young', 'production', 'while', 'die', 'and', 'than', 'came', 'so', 'cliche', 'show', 'entertainment', 'a

Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

Max = 2494
Min = 11


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [8]:
import numpy as np 
np.max(lenghts), np.min(lenghts)

(2494, 11)

In [12]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test

x_train_pad = sequence.pad_sequences(X_train,maxlen = max_words)
X_test_pad = sequence.pad_sequences(X_test, maxlen = max_words)

### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [23]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
model = Sequential()
model.add(Embedding(vocabulary_size, 32, input_length = max_words))
model.add(LSTM(units = 100, activation = 'tanh'))
model.add(Dense(1, activation = 'sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

 213,301 trainable parameters

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [24]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [25]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 3

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)

# TODO: Train your model
model.fit(x_train_pad, y_train, validation_split = 0.1, epochs = num_epochs, batch_size = batch_size)


Train on 22500 samples, validate on 2500 samples
Epoch 1/3
22500/22500 [==============================] - 480s 21ms/step - loss: 0.4706 - acc: 0.7612 - val_loss: 0.4205 - val_acc: 0.8148
Epoch 2/3
22500/22500 [==============================] - 484s 21ms/step - loss: 0.3429 - acc: 0.8544 - val_loss: 0.3366 - val_acc: 0.8644
Epoch 3/3
22500/22500 [==============================] - 481s 21ms/step - loss: 0.2626 - acc: 0.8966 - val_loss: 0.3885 - val_acc: 0.8204


In [27]:
import os 

In [33]:
# Save your model, so that you can quickly load it in future (and perhaps resume training)
model_file = "rnn_model.h5"  # HDF5 file
#model.save(os.path.join(cache_dir, model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [32]:
# Evaluate your model on the test set
scores = model.evaluate(X_test_pad, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.8366


#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:
While our Bow + GradientBoosted scored around 81% on our test set, the RNN model with LSTM and embedding layers got around 83% on our test set.

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!